In [1]:
from pathlib import Path

import cv2
import numpy as np

import echonet

In [7]:
ECHONET_DIR = Path("/fs04/vf38/ultrasound/agallon/output/segmentation/all-patients/videos")
FLIPPED_DIR = Path("/fs04/vf38/ultrasound/agallon/output/segmentation/flipped/videos")
ECHONET_INPUT_DIR = Path("/fs03/vf38/ultrasound/echonet-data/Videos")

In [39]:
def find_mask(video):
    # (channels, frames, height, width)

    is_red = (video[0] > 230) # (frames, height, width)
    is_notgreen = (video[1] < 50)
    is_notblue = (video[2] <50)

    # return is_red
    return is_red & is_notgreen & is_notblue


In [ ]:
def find_height(mask):
    max_height = mask.shape[1]

    min_y = max_height - 1
    max_y = 0
    for i in range(max_height):
        row = mask[:, ]


In [50]:
def first_nonzero(arr, axis, invalid_val=-1):
    mask = arr!=0
    return np.where(mask.any(axis=axis), mask.argmax(axis=axis), invalid_val)

In [100]:
def get_heights(mask):
    """Returns array of heights of mask for each frame!"""
    frame_indices, row_indices = np.where(mask.any(axis=1)==True)

    heights = []
    for frame_index in range(mask.shape[0]):
        this_frame = frame_indices == frame_index
        min_row, max_row = (min(row_indices[this_frame]), max(row_indices[this_frame]))
        height = max_row - min_row
        heights.append(height)

    heights = np.array(heights)
    return heights

In [104]:
for i, video_fp in enumerate(ECHONET_DIR.iterdir()):
    if i > 1:
        break
    echonet_segment_video = echonet.utils.loadvideo(str(video_fp))
    echonet_segment_video = echonet_segment_video[:, :, 0:112, 112:]
    left_only_mask = find_mask(echonet_segment_video)

    video_name = video_fp.name
    flipped_video_fp = FLIPPED_DIR / video_name

    flipped_video = echonet.utils.loadvideo(str(flipped_video_fp))
    flipped_video = flipped_video[:, :, 0:112, 112:]
    flipped_video = np.flip(flipped_video, axis=3)
    flipped_mask = find_mask(flipped_video)

    or_mask = left_only_mask | flipped_mask
    and_mask = left_only_mask & flipped_mask
    echonet_video_fp = ECHONET_INPUT_DIR / video_name
    echonet_video = echonet.utils.loadvideo(str(echonet_video_fp))

    # left_only = echonet_mask - and_mask
    # both_only 

    right_only_mask = flipped_mask ^ (left_only_mask & flipped_mask) 

    left_heights = get_heights(left_only_mask)
    right_heights = get_heights(right_only_mask)
    bad_heights = right_heights > 0.8 * left_heights
    print(bad_heights)
    # print((bad_heights == False).sum())
    # print(right_heights > left_heights)
    # print(left_heights)
    # print(right_heights)


    echonet_video[0, right_only_mask] = 0
    echonet_video[1, right_only_mask] = 0
    echonet_video[2, right_only_mask] = 255

    echonet_video[0, left_only_mask] = 255
    echonet_video[1, left_only_mask] = 0
    echonet_video[2, left_only_mask] = 0

    echonet_video[0, and_mask] = 0
    echonet_video[1, and_mask] = 255
    echonet_video[2, and_mask] = 0
    
    echonet_video[0, bad_heights, 0, :] = 255
    echonet_video[1, bad_heights, 0, :] = 0
    echonet_video[2, bad_heights, 0, :] = 0
    echonet_video[0, bad_heights, -1, :] = 255
    echonet_video[1, bad_heights, -1, :] = 0
    echonet_video[2, bad_heights, -1, :] = 0

    echonet.utils.savevideo("blah.avi", echonet_video, 30)

    # Just Left segmentation = RED (ORIGINAL VIDEO)
    # Just Right segmentation = BLUE ()
    # Both segmentations = GREEN (AND)


[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False  True  True
 False False  True  True False  True  True  True  True  True False False
  True  True  True  True  True  True  True  True  True False False False
  True  True  True  True  True False False False False False False False
 False False False False False False False False False False False False
 False False False False False False  True  True  True  True  True  True
  True  True  True  True  True False False False Fa

In [20]:
print(flipped_video.shape)
print(flipped_mask.shape)
print(or_mask.shape)

(3, 224, 112, 112)
(224, 112, 112)
(224, 112, 112)


In [48]:
flipped_mask.shape

(188, 112, 112)

In [15]:

# frame_index = 0
# while frame_index < len(echonet_video[1]):
#     frame = echonet_video[frame_index]

#     # Display the resulting frame
#     cv2.imshow('Frame',frame)
 
#     # Press Q on keyboard to  exit
#     if cv2.waitKey(25) & 0xFF == ord('q'):
#         break
    
#     frame_index += 1


# cv2.destroyAllWindows()

: 

: 